In [1]:
from sklearn.model_selection import KFold
import pickle
import pandas as pd
import os.path
file_name = 'resampled_2x5_cross_validation'
if os.path.isfile(file_name+'.pickle'):
    with open(file_name+'.pickle', 'rb') as handle:
        train_folds,test_folds = pickle.load(handle)
                                            

In [2]:
import transformer_lstr as tr
import torch.nn as nn
import torch
class feature_encoder_1d(nn.Module):
    def __init__(self, input_len, d_model):
        super(feature_encoder_1d, self).__init__()
        self.input_len = input_len
        self.d_model = d_model
        self.feature_encoder = nn.Linear(self.input_len, self.input_len * self.d_model)
        
    def forward(self, encoded_features):
        _b, _l, _c = encoded_features.shape
        # assert _l == self.input_len and _c == 1
        encoded_features = encoded_features.flatten(1,2) # (Bx64x1) -> (Bx64)
        encoded_features = self.feature_encoder(encoded_features) # (B x 64) -> (B x 64*d_model)
        encoded_features = encoded_features.view(_b, _l, self.d_model) # (B x 64*d_model32) -> (Bx64xd_model32)
        return encoded_features
        
        
class SpectrumTransformer(nn.Module):
    def __init__(self):
        super(SpectrumTransformer, self).__init__()
        
        self.training = False

        # Decoder configs
        self.input_len = 64
        self.d_model_init = 1
        
        self.d_model = 16
        self.d_val = 250
        self.num_heads = 1
        self.dim_feedforward = 256
        self.dropout = 0.
        self.activation = 'relu'
        
        self.embedding_len = [
            [64, 4, True],[64, 2, True], [250, 1, True]
        ]
        self.num_classes = 250

        # Build position encoding
        self.pos_encoding = tr.PositionalEncoding(self.d_model, self.dropout)

        # decoder modules
        # self.keys = nn.Parameter(torch.ones(self.input_len, self.d_model), requires_grad=True).to(device)
        self.keys = nn.Embedding(self.input_len, self.d_model)
        
        # self.values = nn.Parameter(rand(self.input_len, self.d_val), requires_grad=True).to(device)
        # enc_layer = tr.TransformerDecoderLayer(
        #                 self.d_model, self.num_heads, self.dim_feedforward,
        #                 self.dropout, self.activation
        #             )
        # self.enc_modules.append(
        #     tr.TransformerDecoder(
        #         enc_layer, param[1], tr.layer_norm(self.d_model, param[2])
        #     )
        # )
        
        self.transformer = tr.Transformer(d_model=self.d_model, nhead=self.num_heads, num_encoder_layers=1,
                 num_decoder_layers=1, dim_feedforward=self.dim_feedforward, dropout=self.dropout,
                 activation='relu', custom_encoder=None, custom_decoder=None)


        # Build classifier
        self.feature_encoder = feature_encoder_1d(self.input_len, self.d_model)
        
        self.feature_decoder = nn.Sequential(
            nn.Linear(self.d_model, 64),
            # nn.Sigmoid(),
        )
        
        self.feature_decoder2 = nn.Sequential(
            nn.Linear(self.d_model*self.input_len, 250),
            nn.Sigmoid(),
        )
    
    def create_keys(self, keys, encoded_features):
        _b, _l, _c = encoded_features.shape
        keys = keys.expand(_b, -1, -1)
        return keys

    def forward(self, encoded_features, memory_key_padding_mask=None):
        encoded_features = self.feature_encoder(encoded_features) # (B x 64) -> (B x 64*d_model32)
        encoded_features = encoded_features.transpose(0, 1) # (Bx64xd_model32) -> (64xBxd_model32)
        q = self.pos_encoding(encoded_features)
        # print(self.create_keys(self.keys, encoded_features).shape, q.shape)
        spectrum = self.transformer(self.create_keys(self.keys.weight.unsqueeze(1).repeat(1, encoded_features.shape[1], 1), encoded_features), q)
        # spectrum = self.transformer(self.create_keys(self.keys, encoded_features), q)
        # kkk = self.keys.weight.unsqueeze(1).repeat(1, encoded_features.shape[1], 1)
        # print(torch.max(kkk), torch.min(kkk))
        # spectrum = self.feature_decoder(spectrum)
        # print(spectrum.shape)
        spectrum = spectrum.transpose(0, 1).flatten(1,2)
        # print(spectrum.shape)
        spectrum = self.feature_decoder2(spectrum)
        # print(spectrum.shape)
        return spectrum[:,:,None]

In [3]:
from torch.utils.data import DataLoader
from torch import optim
import numpy as np
class CalculateMSE():
    def __init__(self, device,n_epochs,batch_size ):
        super().__init__()
        self.net = SpectrumTransformer().to(device)
        #initialize some constants
        self.batch_size = 32
        self.learning_rate = 1e-4
        self.n_epochs = n_epochs
        self.net.apply(self.weights_init)   
        self.device=device
    def weights_init(self,layer):
        if type(layer) == nn.Linear:
            nn.init.orthogonal_(layer.weight)
    def get_mse(self,train_data, train_label, test_data, test_label):
        train_set = torch.utils.data.TensorDataset(
            torch.Tensor(train_data), 
            torch.Tensor(train_label))
        val_set = torch.utils.data.TensorDataset(
            torch.Tensor(test_data), 
            torch.Tensor(test_label))
        loader_args = dict(batch_size=self.batch_size)
        train_loader = DataLoader(train_set, shuffle=True, drop_last=True, **loader_args)
        val_loader = DataLoader(val_set, shuffle=True, drop_last=True, **loader_args)
        tloss = []
        vloss = []
        criterion = nn.MSELoss()
        optimizer = optim.Adam(self.net.parameters(), lr=self.learning_rate) # weight_decay=0
        for epoch in range(0, self.n_epochs):
            epoch_train_loss=[]
            self.net.train()
            for i, data in enumerate(train_loader, 0):
                inputs, label = data
                y_pred = self.net(inputs.to(self.device).unsqueeze(2))
                loss = criterion(y_pred, label.to(self.device).unsqueeze(2))
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                epoch_train_loss.append(loss.item())
            tloss.append(np.mean(epoch_train_loss))
            epoch_loss=[]
            self.net.eval()
            for i, data in enumerate(val_loader, 0):
                with torch.no_grad():
                    inputs1, label1 = data
                    y_pred1 = self.net(inputs1.to(self.device).unsqueeze(2))
                    loss1 = criterion(y_pred1, label1.to(self.device).unsqueeze(2))
                    epoch_loss.append(loss1.item())
            vloss.append(np.mean(epoch_loss))
        return np.min(vloss), self.net


In [4]:
from pathlib import Path
from scipy import stats,spatial
from dtw import *
import torch
import numpy as np
import dcor
n_epochs=3000
batch_size=32
PATH = 'saved_model/Transformer_combined_poisson_10percent/'
Path(PATH).mkdir(parents=True, exist_ok=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
losses = []
def add_noise(inputs,inputs2, a = 0.02, std = 0.02, sequence_length=64, noise_seed=None):
    if noise_seed is not None:
        np.random.seed(noise_seed)
    noise = np.random.normal(0,std, size = (inputs.shape[0], sequence_length)).astype(np.float32)
    noise2 = np.random.normal(0,std, size = (inputs2.shape[0], sequence_length)).astype(np.float32)
    #noise = poisson.rvs(mu, size=(inputs.shape[0], sequence_length)).astype(np.float32)
    
    nsd = a*np.random.poisson(inputs/a).astype(np.float32)
    nsd2 = a*np.random.poisson(inputs2/a).astype(np.float32)
    
    # Calculate the absolute error between nsd and original inputs
    absolute_error = np.abs(nsd - inputs)
    absolute_error2 = np.abs(nsd2 - inputs2)
    # Sum up the absolute errors
    total_error = np.sum(absolute_error)
    total_error2 = np.sum(absolute_error2)
    # Sum of the original data
    total_original = np.sum(inputs)
    total_original2 = np.sum(inputs2)

    # Calculate the noise ratio
    noise_ratio = (total_error+total_error2) / (total_original+total_original2)
    
    return nsd, nsd2, noise_ratio
    

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [7]:
correlation_losses = []
model = SpectrumTransformer().to(device)
print("noise ratio: 10%")
def calculate_correlation(model, test_data, test_label):
    test_data_tensor = torch.tensor(test_data, dtype=torch.float32).unsqueeze(2).to(device)
    construction = model(test_data_tensor).squeeze(2).detach().cpu().numpy()
   
    # Pearson
    pearson_coefs = []
    pearson_ps = []
    
    # Kendall
    kendall_coefs = []
    kendall_ps = []
    
    # Spearman
    spearman_coefs = []
    spearman_ps = []
    # Distance Correlation
    distance_corr = []
    #DTW distance
    alignment = []

    #absolute_error
    abs_err = []

    for i in range(test_label.shape[0]):
        x1 = construction[i,:]
        x2 = test_label[i,:]
        
        res = stats.pearsonr(x1, x2)
        pearson_coefs.append(res[0])
        pearson_ps.append(res[1])
        
        res = stats.kendalltau(x1, x2)
        kendall_coefs.append(res[0])
        kendall_ps.append(res[1])
        
        res = stats.spearmanr(x1, x2)
        spearman_coefs.append(res[0])
        spearman_ps.append(res[1])
        
        distance_corr.append(dcor.distance_correlation(x1,x2))
        
        alignment.append(dtw(x1, x2, distance_only=True).distance)
        abs_err.append(abs(x1-x2))
        
    correlation_results = {
        'pearson': (pearson_coefs, pearson_ps),
        'kendall': (kendall_coefs, kendall_ps),
        'spearman': (spearman_coefs, spearman_ps),
        'DTW': alignment,
        'Absolute Error': abs_err,
        'Distance Correlation': distance_corr
    }

    return correlation_results


for i,(train,test) in enumerate(zip(train_folds,test_folds)):
    train_data, train_label= train[0],train[1]
    test_data, test_label= test[0],test[1]
    train_data,test_data,noise_ratio = add_noise(train_data,test_data, a=0.0065,std = 0.05, sequence_length=64, noise_seed=i)
    mdl_name = PATH + 'model' + str(i)
    model.load_state_dict(torch.load(mdl_name))
    model.eval()
    
    correlation_loss = calculate_correlation(model, test_data, test_label)
    correlation_losses.append(correlation_loss)
for key in correlation_losses[0].keys():
    print(key)
    if key=='Absolute Error':
        errors = []
        for d in correlation_losses:
            errors+=np.concatenate(d[key]).ravel().tolist()
        #percentile
        percentiles = [5, 50, 90, 95, 99]
        for p in percentiles:
            print(p)
            print(np.percentile(errors, p))
    else:
        stat, p = [], []
        for d in correlation_losses:
            if key=='DTW' or key=='Distance Correlation':
                stat+=d[key]
            else:
                stat+=d[key][0]
                p+=d[key][1]
        print(np.mean(stat),np.std(stat))
        if len(p)>0:
            print(np.mean(p),np.std(p))

noise ratio: 10%
pearson
0.913857702948878 0.2032878334113308
0.0038291122945179417 0.05023791263279459
kendall
0.6684456876331049 0.19660450060556828
0.0049231469611440305 0.05655665511150282
spearman
0.8153772845486946 0.21081843704454764
0.004461289017804344 0.04788967548905703
DTW
7.5806376818915835 10.42861225643274
Absolute Error
5
0.0008187264634811645
50
0.01674323415125949
90
0.13094186160660293
95
0.19676702841573132
99
0.3788347323323866
Distance Correlation
0.9327480840651144 0.12787289559283185


In [8]:
PATH = 'saved_model/Transformer_combined_poisson_15percent/'
correlation_losses = []
model = SpectrumTransformer().to(device)
print("noise ratio: 15%")
def calculate_correlation(model, test_data, test_label):
    test_data_tensor = torch.tensor(test_data, dtype=torch.float32).unsqueeze(2).to(device)
    construction = model(test_data_tensor).squeeze(2).detach().cpu().numpy()
   
    # Pearson
    pearson_coefs = []
    pearson_ps = []
    
    # Kendall
    kendall_coefs = []
    kendall_ps = []
    
    # Spearman
    spearman_coefs = []
    spearman_ps = []
    # Distance Correlation
    distance_corr = []
    #DTW distance
    alignment = []

    #absolute_error
    abs_err = []

    for i in range(test_label.shape[0]):
        x1 = construction[i,:]
        x2 = test_label[i,:]
        
        res = stats.pearsonr(x1, x2)
        pearson_coefs.append(res[0])
        pearson_ps.append(res[1])
        
        res = stats.kendalltau(x1, x2)
        kendall_coefs.append(res[0])
        kendall_ps.append(res[1])
        
        res = stats.spearmanr(x1, x2)
        spearman_coefs.append(res[0])
        spearman_ps.append(res[1])
        
        distance_corr.append(dcor.distance_correlation(x1,x2))
        
        alignment.append(dtw(x1, x2, distance_only=True).distance)
        abs_err.append(abs(x1-x2))
        
    correlation_results = {
        'pearson': (pearson_coefs, pearson_ps),
        'kendall': (kendall_coefs, kendall_ps),
        'spearman': (spearman_coefs, spearman_ps),
        'DTW': alignment,
        'Absolute Error': abs_err,
        'Distance Correlation': distance_corr
    }

    return correlation_results


for i,(train,test) in enumerate(zip(train_folds,test_folds)):
    train_data, train_label= train[0],train[1]
    test_data, test_label= test[0],test[1]
    train_data,test_data,noise_ratio = add_noise(train_data,test_data, a=0.0145, std = 0.05, sequence_length=64, noise_seed=i)
    mdl_name = PATH + 'model' + str(i)
    model.load_state_dict(torch.load(mdl_name))
    model.eval()
    
    correlation_loss = calculate_correlation(model, test_data, test_label)
    correlation_losses.append(correlation_loss)
for key in correlation_losses[0].keys():
    print(key)
    if key=='Absolute Error':
        errors = []
        for d in correlation_losses:
            errors+=np.concatenate(d[key]).ravel().tolist()
        #percentile
        percentiles = [5, 50, 90, 95, 99]
        for p in percentiles:
            print(p)
            print(np.percentile(errors, p))
    else:
        stat, p = [], []
        for d in correlation_losses:
            if key=='DTW' or key=='Distance Correlation':
                stat+=d[key]
            else:
                stat+=d[key][0]
                p+=d[key][1]
        print(np.mean(stat),np.std(stat))
        if len(p)>0:
            print(np.mean(p),np.std(p))

noise ratio: 15%
pearson
0.849768043860994 0.3024671068183425
0.007195399400919641 0.06382398453001144
kendall
0.6239854845157964 0.23912908230690658
0.00881442698578495 0.07181724033553852
spearman
0.7657017753798179 0.280015849928557
0.00901040090619814 0.07238784403273593
DTW
10.93314131611527 15.342339634728571
Absolute Error
5
0.0009476221063285909
50
0.023224660512460304
90
0.18014033027245954
95
0.2817689595369097
99
0.5419589644572956
Distance Correlation
0.8953778469808792 0.17400404055236648


In [9]:
PATH = 'saved_model/Transformer_combined_poisson_20percent/'
correlation_losses = []
model = SpectrumTransformer().to(device)
print("noise ratio: 20%")
def calculate_correlation(model, test_data, test_label):
    test_data_tensor = torch.tensor(test_data, dtype=torch.float32).unsqueeze(2).to(device)
    construction = model(test_data_tensor).squeeze(2).detach().cpu().numpy()
   
    # Pearson
    pearson_coefs = []
    pearson_ps = []
    
    # Kendall
    kendall_coefs = []
    kendall_ps = []
    
    # Spearman
    spearman_coefs = []
    spearman_ps = []
    # Distance Correlation
    distance_corr = []
    #DTW distance
    alignment = []

    #absolute_error
    abs_err = []

    for i in range(test_label.shape[0]):
        x1 = construction[i,:]
        x2 = test_label[i,:]
        
        res = stats.pearsonr(x1, x2)
        pearson_coefs.append(res[0])
        pearson_ps.append(res[1])
        
        res = stats.kendalltau(x1, x2)
        kendall_coefs.append(res[0])
        kendall_ps.append(res[1])
        
        res = stats.spearmanr(x1, x2)
        spearman_coefs.append(res[0])
        spearman_ps.append(res[1])
        
        distance_corr.append(dcor.distance_correlation(x1,x2))
        
        alignment.append(dtw(x1, x2, distance_only=True).distance)
        abs_err.append(abs(x1-x2))
        
    correlation_results = {
        'pearson': (pearson_coefs, pearson_ps),
        'kendall': (kendall_coefs, kendall_ps),
        'spearman': (spearman_coefs, spearman_ps),
        'DTW': alignment,
        'Absolute Error': abs_err,
        'Distance Correlation': distance_corr
    }

    return correlation_results


for i,(train,test) in enumerate(zip(train_folds,test_folds)):
    train_data, train_label= train[0],train[1]
    test_data, test_label= test[0],test[1]
    train_data,test_data,noise_ratio = add_noise(train_data,test_data, a=0.026,std = 0.05, sequence_length=64, noise_seed=i)
    mdl_name = PATH + 'model' + str(i)
    model.load_state_dict(torch.load(mdl_name))
    model.eval()
    
    correlation_loss = calculate_correlation(model, test_data, test_label)
    correlation_losses.append(correlation_loss)
for key in correlation_losses[0].keys():
    print(key)
    if key=='Absolute Error':
        errors = []
        for d in correlation_losses:
            errors+=np.concatenate(d[key]).ravel().tolist()
        #percentile
        percentiles = [5, 50, 90, 95, 99]
        for p in percentiles:
            print(p)
            print(np.percentile(errors, p))
    else:
        stat, p = [], []
        for d in correlation_losses:
            if key=='DTW' or key=='Distance Correlation':
                stat+=d[key]
            else:
                stat+=d[key][0]
                p+=d[key][1]
        print(np.mean(stat),np.std(stat))
        if len(p)>0:
            print(np.mean(p),np.std(p))

noise ratio: 20%
pearson
0.793287948081437 0.3740841515316291
0.011211217778967298 0.08192452768851952
kendall
0.5828770284046696 0.2731111265954169
0.015567620340598682 0.09607547758588196
spearman
0.7179462695874881 0.33772964431265173
0.013952048538182106 0.0850751276728114
DTW
14.592226178951146 19.390879392354716
Absolute Error
5
0.0011015116513745159
50
0.031753984736293955
90
0.24031586295695243
95
0.3726154444428482
99
0.6334698388080311
Distance Correlation
0.8697815273866714 0.19180988011787217


In [10]:
PATH = 'saved_model/Transformer_combined_poisson_25percent/'
correlation_losses = []
model = SpectrumTransformer().to(device)
print("noise ratio: 25%")
def calculate_correlation(model, test_data, test_label):
    test_data_tensor = torch.tensor(test_data, dtype=torch.float32).unsqueeze(2).to(device)
    construction = model(test_data_tensor).squeeze(2).detach().cpu().numpy()
   
    # Pearson
    pearson_coefs = []
    pearson_ps = []
    
    # Kendall
    kendall_coefs = []
    kendall_ps = []
    
    # Spearman
    spearman_coefs = []
    spearman_ps = []
    # Distance Correlation
    distance_corr = []
    #DTW distance
    alignment = []

    #absolute_error
    abs_err = []

    for i in range(test_label.shape[0]):
        x1 = construction[i,:]
        x2 = test_label[i,:]
        
        res = stats.pearsonr(x1, x2)
        pearson_coefs.append(res[0])
        pearson_ps.append(res[1])
        
        res = stats.kendalltau(x1, x2)
        kendall_coefs.append(res[0])
        kendall_ps.append(res[1])
        
        res = stats.spearmanr(x1, x2)
        spearman_coefs.append(res[0])
        spearman_ps.append(res[1])
        
        distance_corr.append(dcor.distance_correlation(x1,x2))
        
        alignment.append(dtw(x1, x2, distance_only=True).distance)
        abs_err.append(abs(x1-x2))
        
    correlation_results = {
        'pearson': (pearson_coefs, pearson_ps),
        'kendall': (kendall_coefs, kendall_ps),
        'spearman': (spearman_coefs, spearman_ps),
        'DTW': alignment,
        'Absolute Error': abs_err,
        'Distance Correlation': distance_corr
    }

    return correlation_results


for i,(train,test) in enumerate(zip(train_folds,test_folds)):
    train_data, train_label= train[0],train[1]
    test_data, test_label= test[0],test[1]
    train_data,test_data,noise_ratio = add_noise(train_data,test_data, a=0.0403, std = 0.05, sequence_length=64, noise_seed=i)
    mdl_name = PATH + 'model' + str(i)
    model.load_state_dict(torch.load(mdl_name))
    model.eval()
    
    correlation_loss = calculate_correlation(model, test_data, test_label)
    correlation_losses.append(correlation_loss)
for key in correlation_losses[0].keys():
    print(key)
    if key=='Absolute Error':
        errors = []
        for d in correlation_losses:
            errors+=np.concatenate(d[key]).ravel().tolist()
        #percentile
        percentiles = [5, 50, 90, 95, 99]
        for p in percentiles:
            print(p)
            print(np.percentile(errors, p))
    else:
        stat, p = [], []
        for d in correlation_losses:
            if key=='DTW' or key=='Distance Correlation':
                stat+=d[key]
            else:
                stat+=d[key][0]
                p+=d[key][1]
        print(np.mean(stat),np.std(stat))
        if len(p)>0:
            print(np.mean(p),np.std(p))

noise ratio: 25%
pearson
0.7311771257500903 0.42695880406052417
0.01642440054663974 0.09930836681152819
kendall
0.5366107848069058 0.2964976688842507
0.015303860234916745 0.09270055404018361
spearman
0.6672798358539361 0.37315193380311795
0.01628414492905682 0.09305000467338252
DTW
17.84545224496977 21.3956751781105
Absolute Error
5
0.0011922719894318782
50
0.0414582251981395
90
0.3049064935700261
95
0.4394114460908053
99
0.6700467275962447
Distance Correlation
0.8381618535621269 0.21255847000638614


In [11]:
PATH = 'saved_model/Transformer_combined_poisson_30percent/'
correlation_losses = []
model = SpectrumTransformer().to(device)
print("noise ratio: 30%")
def calculate_correlation(model, test_data, test_label):
    test_data_tensor = torch.tensor(test_data, dtype=torch.float32).unsqueeze(2).to(device)
    construction = model(test_data_tensor).squeeze(2).detach().cpu().numpy()
   
    # Pearson
    pearson_coefs = []
    pearson_ps = []
    
    # Kendall
    kendall_coefs = []
    kendall_ps = []
    
    # Spearman
    spearman_coefs = []
    spearman_ps = []
    # Distance Correlation
    distance_corr = []
    #DTW distance
    alignment = []

    #absolute_error
    abs_err = []

    for i in range(test_label.shape[0]):
        x1 = construction[i,:]
        x2 = test_label[i,:]
        
        res = stats.pearsonr(x1, x2)
        pearson_coefs.append(res[0])
        pearson_ps.append(res[1])
        
        res = stats.kendalltau(x1, x2)
        kendall_coefs.append(res[0])
        kendall_ps.append(res[1])
        
        res = stats.spearmanr(x1, x2)
        spearman_coefs.append(res[0])
        spearman_ps.append(res[1])
        
        distance_corr.append(dcor.distance_correlation(x1,x2))
        
        alignment.append(dtw(x1, x2, distance_only=True).distance)
        abs_err.append(abs(x1-x2))
        
    correlation_results = {
        'pearson': (pearson_coefs, pearson_ps),
        'kendall': (kendall_coefs, kendall_ps),
        'spearman': (spearman_coefs, spearman_ps),
        'DTW': alignment,
        'Absolute Error': abs_err,
        'Distance Correlation': distance_corr
    }

    return correlation_results


for i,(train,test) in enumerate(zip(train_folds,test_folds)):
    train_data, train_label= train[0],train[1]
    test_data, test_label= test[0],test[1]
    train_data,test_data,noise_ratio = add_noise(train_data,test_data, a=0.058,std = 0.05, sequence_length=64, noise_seed=i)
    mdl_name = PATH + 'model' + str(i)
    model.load_state_dict(torch.load(mdl_name))
    model.eval()
    
    correlation_loss = calculate_correlation(model, test_data, test_label)
    correlation_losses.append(correlation_loss)
for key in correlation_losses[0].keys():
    print(key)
    if key=='Absolute Error':
        errors = []
        for d in correlation_losses:
            errors+=np.concatenate(d[key]).ravel().tolist()
        #percentile
        percentiles = [5, 50, 90, 95, 99]
        for p in percentiles:
            print(p)
            print(np.percentile(errors, p))
    else:
        stat, p = [], []
        for d in correlation_losses:
            if key=='DTW' or key=='Distance Correlation':
                stat+=d[key]
            else:
                stat+=d[key][0]
                p+=d[key][1]
        print(np.mean(stat),np.std(stat))
        if len(p)>0:
            print(np.mean(p),np.std(p))

noise ratio: 30%
pearson
0.698082371306182 0.4479197773355329
0.015373760251586651 0.09232855848279542
kendall
0.512440810849603 0.31642914672251377
0.01918721442063004 0.10425416849310522
spearman
0.6363106522144729 0.40362344608537726
0.023772063947633978 0.12089169177562441
DTW
20.131477354088865 22.305294268803046
Absolute Error
5
0.0014171600752861616
50
0.050062216897538314
90
0.3384157904967167
95
0.4705390862181813
99
0.6832578413106442
Distance Correlation
0.8215318189033437 0.21695698277189315
